In [140]:
import pandas as pd
import numpy as np
import xarray as xr
import calliope

In [141]:
# Load your weather data
df = pd.read_excel("WeatherData.xlsx")

In [142]:
df.head()

,Unnamed: 0,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,Q,SQ,VV,N,U,P
0,0,330,20240101,1,210,120,120,180,75,NaN,61,0,0,NaN,NaN,91,9951
1,1,330,20240101,2,210,90,100,150,74,NaN,62,0,0,NaN,NaN,91,9958
2,2,330,20240101,3,240,100,110,160,92,NaN,59,0,0,NaN,NaN,79,9959
3,3,330,20240101,4,240,110,110,190,91,NaN,59,0,0,NaN,NaN,80,9966
4,4,330,20240101,5,240,100,100,150,95,NaN,54,0,0,NaN,NaN,76,9973


In [143]:
# Ensure 'HH' is properly formatted
df["HH"] = df["HH"].astype(str).str.zfill(2)

In [144]:
# Handle '24' hour values by shifting to next day
def parse_datetime(row):
    hour = int(row["HH"])
    date = pd.to_datetime(str(row["YYYYMMDD"]), format="%Y%m%d")
    if hour == 24:
        return date + pd.Timedelta(days=1)
    else:
        return date + pd.Timedelta(hours=hour)

df["datetime"] = df.apply(parse_datetime, axis=1)
df.set_index("datetime", inplace=True)

In [145]:
# Convert raw values
df["temperature_C"] = df["T"] / 10
df["global_irradiance_Wm2"] = (df["Q"] * 10000) / 3600
df["sunshine_hours"] = df["SQ"] * 0.1

In [146]:
# Keep only relevant columns
df_clean = df[["temperature_C", "global_irradiance_Wm2", "sunshine_hours"]]

In [147]:
# Convert to xarray
ds_weather = xr.Dataset.from_dataframe(df_clean)

In [148]:
# Preview
ds_weather

<xarray.Dataset>
Dimensions:                (datetime: 744)
Coordinates:
  * datetime               (datetime) datetime64[ns] 2024-01-01T01:00:00 ... ...
Data variables:
    temperature_C          (datetime) float64 7.5 7.4 9.2 9.1 ... 7.8 7.9 9.0
    global_irradiance_Wm2  (datetime) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    sunshine_hours         (datetime) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [149]:
irradiance_kwh = df_clean["global_irradiance_Wm2"] / 1000   # Convert W/m² to kWh/m²



In [150]:
# Save CSV with location name as header
irradiance_kwh.to_csv("solar_model/time_series/irradiance.csv", header=["nl_site"])



In [151]:
pd.read_csv("solar_model/time_series/irradiance.csv", header=None).head()

,0,1
0,datetime,nl_site
1,2024-01-01 01:00:00,0.0
2,2024-01-01 02:00:00,0.0
3,2024-01-01 03:00:00,0.0
4,2024-01-01 04:00:00,0.0


In [152]:
df.to_csv("solar_model/time_series/irradiance_cleaned.csv")


In [153]:
pd.read_csv("solar_model/time_series/irradiance_cleaned.csv", header=None).head(10)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,datetime,Unnamed: 0,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,...,TD,Q,SQ,VV,N,U,P,temperature_C,global_irradiance_Wm2,sunshine_hours
1,2024-01-01 01:00:00,0,330,20240101,01,210,120,120,180,75,...,61,0,0,NaN,NaN,91,9951,7.5,0.0,0.0
2,2024-01-01 02:00:00,1,330,20240101,02,210,90,100,150,74,...,62,0,0,NaN,NaN,91,9958,7.4,0.0,0.0
3,2024-01-01 03:00:00,2,330,20240101,03,240,100,110,160,92,...,59,0,0,NaN,NaN,79,9959,9.2,0.0,0.0
4,2024-01-01 04:00:00,3,330,20240101,04,240,110,110,190,91,...,59,0,0,NaN,NaN,80,9966,9.1,0.0,0.0
5,2024-01-01 05:00:00,4,330,20240101,05,240,100,100,150,95,...,54,0,0,NaN,NaN,76,9973,9.5,0.0,0.0
6,2024-01-01 06:00:00,5,330,20240101,06,250,120,120,170,92,...,48,0,0,NaN,NaN,73,9978,9.2,0.0,0.0
7,2024-01-01 07:00:00,6,330,20240101,07,260,130,120,180,91,...,48,0,0,NaN,NaN,74,9989,9.1,0.0,0.0
8,2024-01-01 08:00:00,7,330,20240101,08,260,130,130,180,92,...,46,0,0,NaN,NaN,73,10001,9.2,0.0,0.0
9,2024-01-01 09:00:00,8,330,20240101,09,260,130,140,190,90,...,48,3,0,NaN,NaN,74,10012,9.0,8.333333333333334,0.0


In [154]:
df = pd.read_csv("solar_model/time_series/irradiance_cleaned.csv", index_col="datetime", parse_dates=True)

In [155]:
# Check row counts per day
daily_counts = df.groupby(df.index.date).size()
print(daily_counts.value_counts())  # Should show 24s only

24    30
23     1
1      1
dtype: int64


In [156]:
# To find problematic days
print(daily_counts[daily_counts != 24])


2024-01-01    23
2024-02-01     1
dtype: int64


In [157]:
import calliope

In [158]:
# Load and inspect the CSV
df = pd.read_csv("solar_model/time_series/irradiance_cleaned.csv")

# Display first few rows and column names
print(df.columns)
print(df.head())

Index(['datetime', 'Unnamed: 0', 'STN', 'YYYYMMDD', 'HH', 'DD', 'FH', 'FF',
       'FX', 'T', 'T10N', 'TD', 'Q', 'SQ', 'VV', 'N', 'U', 'P',
       'temperature_C', 'global_irradiance_Wm2', 'sunshine_hours'],
      dtype='object')
              datetime  Unnamed: 0  STN  YYYYMMDD  HH   DD   FH   FF   FX   T  \
0  2024-01-01 01:00:00           0  330  20240101   1  210  120  120  180  75   
1  2024-01-01 02:00:00           1  330  20240101   2  210   90  100  150  74   
2  2024-01-01 03:00:00           2  330  20240101   3  240  100  110  160  92   
3  2024-01-01 04:00:00           3  330  20240101   4  240  110  110  190  91   
4  2024-01-01 05:00:00           4  330  20240101   5  240  100  100  150  95   

   ...  TD  Q  SQ  VV   N   U     P  temperature_C  global_irradiance_Wm2  \
0  ...  61  0   0 NaN NaN  91  9951            7.5                    0.0   
1  ...  62  0   0 NaN NaN  91  9958            7.4                    0.0   
2  ...  59  0   0 NaN NaN  79  9959            9.2  

In [159]:
df.rename(columns={'global_irradiance_Wm2': 'irradiance'}, inplace=True)
df.to_csv("solar_model/time_series/irradiance_cleaned.csv", index=False)

In [160]:
# Load and inspect the CSV
df = pd.read_csv("solar_model/time_series/irradiance_cleaned.csv")

# Display first few rows and column names
print(df.columns)
print(df.head())

Index(['datetime', 'Unnamed: 0', 'STN', 'YYYYMMDD', 'HH', 'DD', 'FH', 'FF',
       'FX', 'T', 'T10N', 'TD', 'Q', 'SQ', 'VV', 'N', 'U', 'P',
       'temperature_C', 'irradiance', 'sunshine_hours'],
      dtype='object')
              datetime  Unnamed: 0  STN  YYYYMMDD  HH   DD   FH   FF   FX   T  \
0  2024-01-01 01:00:00           0  330  20240101   1  210  120  120  180  75   
1  2024-01-01 02:00:00           1  330  20240101   2  210   90  100  150  74   
2  2024-01-01 03:00:00           2  330  20240101   3  240  100  110  160  92   
3  2024-01-01 04:00:00           3  330  20240101   4  240  110  110  190  91   
4  2024-01-01 05:00:00           4  330  20240101   5  240  100  100  150  95   

   ...  TD  Q  SQ  VV   N   U     P  temperature_C  irradiance  sunshine_hours  
0  ...  61  0   0 NaN NaN  91  9951            7.5         0.0             0.0  
1  ...  62  0   0 NaN NaN  91  9958            7.4         0.0             0.0  
2  ...  59  0   0 NaN NaN  79  9959            9.2 

In [161]:
pd.read_csv("solar_model/time_series/irradiance_cleaned.csv", header=None).head(10)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,datetime,Unnamed: 0,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,...,TD,Q,SQ,VV,N,U,P,temperature_C,irradiance,sunshine_hours
1,2024-01-01 01:00:00,0,330,20240101,1,210,120,120,180,75,...,61,0,0,NaN,NaN,91,9951,7.5,0.0,0.0
2,2024-01-01 02:00:00,1,330,20240101,2,210,90,100,150,74,...,62,0,0,NaN,NaN,91,9958,7.4,0.0,0.0
3,2024-01-01 03:00:00,2,330,20240101,3,240,100,110,160,92,...,59,0,0,NaN,NaN,79,9959,9.2,0.0,0.0
4,2024-01-01 04:00:00,3,330,20240101,4,240,110,110,190,91,...,59,0,0,NaN,NaN,80,9966,9.1,0.0,0.0
5,2024-01-01 05:00:00,4,330,20240101,5,240,100,100,150,95,...,54,0,0,NaN,NaN,76,9973,9.5,0.0,0.0
6,2024-01-01 06:00:00,5,330,20240101,6,250,120,120,170,92,...,48,0,0,NaN,NaN,73,9978,9.2,0.0,0.0
7,2024-01-01 07:00:00,6,330,20240101,7,260,130,120,180,91,...,48,0,0,NaN,NaN,74,9989,9.1,0.0,0.0
8,2024-01-01 08:00:00,7,330,20240101,8,260,130,130,180,92,...,46,0,0,NaN,NaN,73,10001,9.2,0.0,0.0
9,2024-01-01 09:00:00,8,330,20240101,9,260,130,140,190,90,...,48,3,0,NaN,NaN,74,10012,9.0,8.333333333333334,0.0


In [162]:
df = pd.read_csv("solar_model/time_series/irradiance_cleaned.csv")

In [163]:
df['datetime'] = pd.to_datetime(df['datetime'])


In [164]:
# Set datetime as index and fill missing hours (resample to hourly)
df = df.set_index('datetime').resample('1H').mean().interpolate()

In [165]:
# Save cleaned file
df.to_csv("solar_model/time_series/irradiance_cleaned.csv")

In [174]:
df.head(10)

,Unnamed: 0,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,Q,SQ,VV,N,U,P,temperature_C,irradiance,sunshine_hours
datetime,,,,,,,,,,,,,,,,,,,,
2024-01-01 01:00:00,0.0,330.0,20240101.0,1.0,210.0,120.0,120.0,180.0,75.0,NaN,61.0,0.0,0.0,NaN,NaN,91.0,9951.0,7.5,0.000000,0.0
2024-01-01 02:00:00,1.0,330.0,20240101.0,2.0,210.0,90.0,100.0,150.0,74.0,NaN,62.0,0.0,0.0,NaN,NaN,91.0,9958.0,7.4,0.000000,0.0
2024-01-01 03:00:00,2.0,330.0,20240101.0,3.0,240.0,100.0,110.0,160.0,92.0,NaN,59.0,0.0,0.0,NaN,NaN,79.0,9959.0,9.2,0.000000,0.0
2024-01-01 04:00:00,3.0,330.0,20240101.0,4.0,240.0,110.0,110.0,190.0,91.0,NaN,59.0,0.0,0.0,NaN,NaN,80.0,9966.0,9.1,0.000000,0.0
2024-01-01 05:00:00,4.0,330.0,20240101.0,5.0,240.0,100.0,100.0,150.0,95.0,NaN,54.0,0.0,0.0,NaN,NaN,76.0,9973.0,9.5,0.000000,0.0
2024-01-01 06:00:00,5.0,330.0,20240101.0,6.0,250.0,120.0,120.0,170.0,92.0,71.000000,48.0,0.0,0.0,NaN,NaN,73.0,9978.0,9.2,0.000000,0.0
2024-01-01 07:00:00,6.0,330.0,20240101.0,7.0,260.0,130.0,120.0,180.0,91.0,71.333333,48.0,0.0,0.0,NaN,NaN,74.0,9989.0,9.1,0.000000,0.0
2024-01-01 08:00:00,7.0,330.0,20240101.0,8.0,260.0,130.0,130.0,180.0,92.0,71.666667,46.0,0.0,0.0,NaN,NaN,73.0,10001.0,9.2,0.000000,0.0
2024-01-01 09:00:00,8.0,330.0,20240101.0,9.0,260.0,130.0,140.0,190.0,90.0,72.000000,48.0,3.0,0.0,NaN,NaN,74.0,10012.0,9.0,8.333333,0.0


In [177]:
model = calliope.Model("solar_model/model.yaml")

C:\Users\csten\anaconda3\envs\calliope-env\lib\site-packages\calliope\preprocess\model_run.py:249: FutureWarning:

There will be no default cost class for the objective function in v0.7.0 (currently "monetary" with a weight of 1). Explicitly specify the cost class(es) you would like to use under `run.objective_options.cost_class`. E.g. `{"monetary": 1}` to replicate the current default.

C:\Users\csten\anaconda3\envs\calliope-env\lib\site-packages\calliope\exceptions.py:60: ModelWarning:

Possible issues found during model processing:
 * Cost classes `{'monetary'}` are defined in the objective options but not defined elsewhere in the model. They will be ignored in the objective function.

C:\Users\csten\anaconda3\envs\calliope-env\lib\site-packages\calliope\preprocess\checks.py:767: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this